# Oil Production

In [1]:
# DATA MANIPULATION, Visualization, and stats
import pandas as pd
from glob import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Your current working directory should be ./mattwagon/oil_production_prediction/notebooks
os.getcwd()

In [3]:
# This filepath should work for all of us
# I have added 'raw_data' to .gitignore so it should never be pushed to the repo
path = os.path.join('..', 'data')

In [4]:
all_files = glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [5]:
df.shape

In [6]:
df.dtypes.Date

In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].dtype
df.Date[0]

In [8]:
# Removes the timezone aspect which makes the column easier to read but changes the date
df['Date'] = df['Date'].dt.tz_convert(None)
df['Date'].dtype
df.Date[0]

In [9]:
sorted_nan = df.isnull().sum().sort_values(ascending=False)
sorted_nan

In [10]:
features_drop = []
for feature, value in dict(sorted_nan).items():
    if value == len(df):
        features_drop.append(feature)

In [11]:
df.drop(columns=features_drop, inplace = True)

In [12]:
dic = {}
for feature, value in dict(sorted_nan).items():
    if value < len(df) and value > 0:
        dic[feature] = value

In [13]:
dic_2 = {}

for feature, value in dic.items():
    dic_2[feature] = round(value/len(df),2) 
    if dic_2[feature] > 0.3:
        features_drop.append(feature)

In [14]:
features_to_impute_check = []

for feature, value in dic_2.items():
    if value < 0.3:
        features_to_impute_check.append(feature)

In [15]:
remove = {"Sand Rate", "MPFM NTotal Count Rate", "MPFM N81 Count Rate", "MPFM N356 Count Rate", "MPFM N32 Count Rate", "MPFM GOR", "Downhole Gauge T", "Downhole Gauge P"}
 
features_to_impute = [f for f in features_to_impute_check if f not in remove]

In [16]:
df_to_impute = df[df.columns.intersection(features_to_impute)]

In [17]:
df_to_impute.head(5)

In [18]:
df_to_impute.describe()

In [19]:
# Pairwise correlation
sns.heatmap(df_to_impute.corr(), cmap='RdBu')
plt.show()

A lot of the 'Choke' features seem to be duplicates of each other or at least extremely similar

Qwat and Annulus P features show strong negative correlation towards our target feature

### Let's look into the choke features

In [20]:
# Get list of choke features
choke_cols = [col for col in df_to_impute.columns if 'Choke' in col]
choke_cols

In [ ]:
# Visual look at distribution of choke data to check for any notable differences
f, axes = plt.subplots(2, 3, figsize=(15, 10))
for ax, feature in zip(axes.flat, df_to_impute[choke_cols]):
    sns.histplot(df_to_impute[feature] , color="skyblue", kde=True, ax=ax)

Looks like there is subtle differences in the data

In [ ]:
# Visual check for outliers
f, axes = plt.subplots(2, 3, figsize=(15, 10))
for ax, feature in zip(axes.flat, df_to_impute[choke_cols]):
    sns.boxplot(data=df_to_impute, x=feature, color="skyblue", ax=ax)

No obvious outliers or differences between features

The features look like pairs (Choke Opening calc 1 - Choke Calculated), (Choke Opening - Choke Measured) and (Choke CCR - Choke Opening Calc 2)

We should drop one of each pair to avoid multicollinearity (I think)

Unsure whether the features skewness will affect scaling

In [ ]:
# Regression look at the choke features
f, axes = plt.subplots(2, 3, figsize=(15, 10))
for ax, feature in zip(axes.flat, choke_cols):
    sns.regplot(data=df_to_impute, x=feature, y=df_to_impute['Qoil MPFM corrected'], ax=ax)

### Look at the negative correlated features

In [ ]:
neg_corr_features = df_to_impute[['Qwat MPFM', 'Qwat MPFM corrected', 'Annulus P']]

In [ ]:
# Visual look at distribution of negatively correlated data
f, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, feature in zip(axes.flat, neg_corr_features):
    sns.histplot(df_to_impute[feature] , color="skyblue", kde=True, ax=ax)

In [ ]:
# Visual check for outliers
f, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, feature in zip(axes.flat, neg_corr_features):
    sns.boxplot(data=neg_corr_features, x=feature, color="skyblue", ax=ax)

In [ ]:
# Visual look at the negative correlated features
f, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, feature in zip(axes.flat, neg_corr_features):
    sns.regplot(data=df_to_impute, x=feature, y=df_to_impute['Qoil MPFM corrected'], ax=ax)